In [28]:


import os
import sys
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, parent_dir)

import pickle
import numpy as np
import math
from Camera import Camera
from Frame import Frame
import matplotlib.pyplot as plt
import Utils

%matplotlib qt


path = 'C:/Users/Roni/Documents/gs_input/frames_model.pkl'
path = 'G:/My Drive/Research/gaussian_splatting/gaussian_splatting_input/mov19_2022_03_03//input_data_for_gs/sparse/frames_model.pkl'
path = 'G:/My Drive/Research/gaussian_splatting/gaussian_splatting_input/mov19_2022_03_03/input_data_for_gs/'
path = 'G:/My Drive/Research/gaussian_splatting/gaussian_splatting_input/mov30_2024_11_12_darkan/'
dict_save_path = 'D:/Documents/data_for_gs/fly_gray/dict/frames_model_1447.pkl'

dict_path = f'{path}/frames_model_1447.pkl'
image_path = f'{path}/'
frame = 1447
with open(dict_path,'rb') as f:
    frames = pickle.load(f)

frames_per_cam = [Frame(image_path,frame,cam_num,frames_dict = frames) for cam_num in range(4)]
ew_to_lab = frames_per_cam[0].rotation_matrix_from_vectors(frames_per_cam[0].R[2,:], [0,0,1])
camera_pixel = np.vstack([frame.camera_center_to_pixel_ray(([frame.cm[0],frame.cm[1]])) for frame in  frames_per_cam])
camera_center = np.vstack([frame.X0.T for frame in  frames_per_cam])
cm_point = Utils.triangulate_least_square(camera_center,camera_pixel)


In [29]:
camera_vectors = (camera_center - camera_pixel)/np.array(np.linalg.norm(camera_center - camera_pixel,axis = 1))[:,np.newaxis]

xyz = [np.linspace(0 - 0.003, 0 + 0.003, 100) for coord in cm_point]
rot_mat = np.array([])
x, y, z = np.meshgrid(xyz[0], xyz[1], xyz[2], indexing="ij")
xyz_flat = np.vstack([x.ravel(), y.ravel(), z.ravel()]).T  # Shape (N, 3)

rot_mats = [frames_per_cam[idx].rotation_matrix_from_vectors(vec, [0,0,1]) for idx,vec in enumerate(camera_vectors)]
xyz_rotated = xyz_flat @ ew_to_lab.T  # Shape (N, 3)
xyz_rot = [xyz_flat @ rot_mat.T for rot_mat in rot_mats]

pt_cloud = np.vstack(xyz_rot + cm_point)    

for cam_num in range(0,4,1):
    pt_cloud = Utils.intersection_per_cam(frames_per_cam,cam_num,pt_cloud)

def get_color(camn):
    pts = frames_per_cam[camn].homogenize_coordinate(pt_cloud)
    pt2dv = frames_per_cam[camn].project_on_image(pts)
    return np.array(frames_per_cam[camn].im)[pt2dv[:,0].astype(int),pt2dv[:,1].astype(int)]

color = np.vstack(3*[np.mean([get_color(camn) for camn in range(4)],axis = 0)]).T
frames[frame][2] = [pt_cloud,color]

In [30]:

with open(dict_save_path, 'wb') as f:
    pickle.dump(frames, f)

In [9]:
import Plotters
import plotly.graph_objects as go


Plotters.plot_projections(pt_cloud,frames_per_cam)

# Plotters.plot_projections(np.append(cm_point,1)[np.newaxis],frames_per_cam, homogenize= False)

fig = go.Figure()
Plotters.scatter3d(fig,camera_center,'red',10,'cam',show_colorbar = False)
Plotters.scatter3d(fig,camera_pixel,'blue',10,'pix',show_colorbar = False)
Plotters.scatter3d(fig,pt_cloud,'green',3,'vol',show_colorbar = False)
fig.show()


d:\Documents\model_gaussian_splatting\Plotters.py:70: UserWarning:

No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored

